In [ ]:
!pip3 install pycaret[full] #usar a versão completa

In [ ]:
!pip install unidecode

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode

In [ ]:
df = pd.read_csv('/content/pre-processed_data_COVID19.csv')
df.head()

In [ ]:
df.columns

##  #Importando o Pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
s = setup(
    data = df,
    target = 'Classification',
    use_gpu=True,
    ignore_features=['Source of Infection	', 'Outbreak Associated']
)

In [ ]:
melhor_modelo = compare_models()

In [ ]:
melhor_modelo

In [ ]:
evaluate_model(melhor_modelo)

In [ ]:
#salvando o modelo
save_model(melhor_modelo, 'modelo_2')

In [ ]:
modelo_final = create_model(melhor_modelo)

In [ ]:
previsoes = predict_model(modelo_final, data=df)

# CONSTRUINDO MINHA API

## Usando FastAPI

In [ ]:
!pip install pycaret[full] fastapi

In [ ]:
!pip install uvicorn

In [ ]:
pip install 'uvicorn[standard]'

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import uvicorn

# Carregando o modelo treinado
modelo = joblib.load('/content/modelo_2.pkl')


app = FastAPI()


class DadosEntrada(BaseModel):
    feature1: float
    feature2: float


# Definindo um endpoint
@app.post("/prever/")
async def prever(dados: DadosEntrada):
    dados_para_previsao = [[dados.feature1, dados.feature2]]

    previsao = modelo.predict(dados_para_previsao)[0]

    # Retornando a previsão
    return {"previsao": previsao}

#### outro código para testar, sem exito.


In [ ]:
import pandas as pd
from pycaret.classification import *
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import joblib

In [ ]:
model = load_model('modelo_2')

In [ ]:
app = FastAPI()

class InputData(BaseModel):
    feature1: float
    feature2: float

@app.post("/predict/")
async def predict(data: InputData):
    try:
        input_data = {
            'feature1': data.feature1,
            'feature2': data.feature2
        }


        prediction = predict_model(model, data=input_data)


        return {'prediction': prediction['Label'][0]}
    except Exception as e:
        return {'error': str(e)}

In [ ]:
if __name__ == '__main__':
  uvicorn.run(app, host='0.0.0.0', port=8000)